Serveur chez OVH :
- Mongo DB
- Docker
- Appli de rajout d'élévation

Réunion du 27/10/2022

- Acces MongoDB via MongoDB Compass
- Interpolation : se donner une tolérance. Si le pointage au passage d'un arrêt est trop éloigné des coordonénes gps rajouter un enregistrement par interpolation
- Sauvegarder la donnée nettoyée sous forme brute dans la base MongoDB (collection)
- Dupliquer les lignes avec des destinations finales différentes


In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
file_arrets = "./Optymo/stops&GPS.xlsx"
df_stops = pd.read_excel(file_arrets, sheet_name=0, header=[0])
stop_names = pd.unique(df_stops['stop_name'].sort_values(ascending=True))
stop_names

array(['1er RA', '1re Armée', '1ère Dfl', '4 As', '5 Fontaines',
       '5ème Db', 'Acacias', 'Ackermann', 'Acropole', 'Adour', 'Allaine',
       'Alsace', 'Alstom Etang', 'Altkirch', 'Ancienne Douane',
       'Ancienne Gare', 'Ancienne École', 'Andelnans', 'Andelnans Prés',
       'Angeot', 'Angoulotte', 'Anjoutey', 'Ap90', 'Arcades', 'Argiésans',
       'As de Trèfle', 'Atria', 'Aubépines', 'Autrechêne', 'Auxelles-Bas',
       'Auxelles-Haut', 'Aéroparc', 'Aéroparc 2', 'Aéroparc 3', 'Badevel',
       'Ballinamuck', 'Ballon', 'Bambois', 'Banvillars', 'Barbier',
       'Barrière Blanche', 'Bas du Village', 'Bascule', 'Batterie',
       'Bavilliers', 'Bavière', 'Beaucourt', 'Bellevue', 'Benoît Frachon',
       'Berger', 'Berly', 'Bermont', 'Bessoncourt', 'Beucler',
       'Beurrerie', 'Bichat', 'Blessonniers', 'Blum', 'Bochets', 'Bohn',
       'Bois Joli', 'Bois Sarclés', "Bois d'Essert", 'Bon Bois',
       'Bonnaymé', 'Bonneff', 'Boron', 'Bosmont', 'Botans', 'Boulloche',
       'Bouloy

In [3]:
df_dest = pd.read_csv('./Optymo/destination.csv')
df_dest

,Destination
0,Argiésans
1,Bavilliers
2,Bois Joli
3,Châtenois
4,Cravanche
5,Éloie
6,Froideval
7,Gare TGV
8,Justice
9,Offemont


In [4]:
def concatDf(filenames):
    list_for_concat = [] #instancier la liste
    #former la liste 'list_for_concat' avec tous les dataframes à concatener
    for f in filenames:     
        df_inter = pd.read_csv(f)
        df_inter = df_inter[(df_inter['line']>0)]     
        df_inter = df_inter[(df_inter['line']<=5)]
        df_inter = df_inter[(df_inter['state']!='Non affecté')]
        df_inter = df_inter[(df_inter['state']!='Délocalisé')]
        df_inter = df_inter[(df_inter['state']!='')]
        df_inter['Year'] = f[11:15]
        df_inter['line'] = pd.to_numeric(df_inter['line'], downcast='integer')
        df_inter['early'] = pd.to_numeric(df_inter['early'], errors='coerce')
        df_inter['late'] = pd.to_numeric(df_inter['late'], errors='coerce')
        df_inter = df_inter.dropna()
        df_inter = df_inter.reset_index(drop = True)
        # add to
        list_for_concat.append(df_inter)
        
    df_final = pd.concat(list_for_concat)
    return df_final

def whitespace_remover(dataframe):
    # iterating over the columns
    for i in dataframe.columns:
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
    return dataframe

In [5]:
 def Update_final_destination(df):
    df['destination'].replace(to_replace="1er RA",                   value="Gare TGV", inplace=True)
    df['destination'].replace(to_replace='1re Armée',                value='Valdoie', inplace=True)
    df['destination'].replace(to_replace='Acacias',                  value='Argiésans', inplace=True)
    df['destination'].replace(to_replace='Argésans',                 value='Argiésans', inplace=True)
    df['destination'].replace(to_replace='ARGIESANS',                value='Argiésans', inplace=True)
    df['destination'].replace(to_replace='Argiésians',               value='Argiésans', inplace=True)
    df['destination'].replace(to_replace='BAVILLIERS',               value='Bavilliers', inplace=True)
    df['destination'].replace(to_replace='Berger',                   value='Froideval', inplace=True)
    df['destination'].replace(to_replace='Cerisiers',                value='Offemont', inplace=True)
    df['destination'].replace(to_replace='CHATENOIS',                value='Châtenois', inplace=True)
    df['destination'].replace(to_replace='Chatenois',                value='Châtenois', inplace=True)
    df['destination'].replace(to_replace='CHATENOIS PAR GARE TGV',   value='Châtenois', inplace=True)
    df['destination'].replace(to_replace='Chaume',                   value='Éloie', inplace=True)
    df['destination'].replace(to_replace='CRAVANCHE',                value='Cravanche', inplace=True)
    df['destination'].replace(to_replace='Eloie Chaume',             value='Éloie', inplace=True)
    df['destination'].replace(to_replace='ELOIE',                    value='Éloie', inplace=True)
    df['destination'].replace(to_replace='Froideval par Pépinière',  value='Froideval', inplace=True)
    df['destination'].replace(to_replace='FROIDEVAL PAR PEPINIERE',  value='Froideval', inplace=True)
    df['destination'].replace(to_replace='GARE TGV',                 value='Gare TGV', inplace=True)
    df['destination'].replace(to_replace='Géhant',                   value='Châtenois', inplace=True)
    df['destination'].replace(to_replace='Hauts de Belfort',         value='Justice', inplace=True)
    df['destination'].replace(to_replace='JUSTICE',                  value='Justice', inplace=True)
    df['destination'].replace(to_replace='LA DAME',                  value='Bavilliers', inplace=True)
    df['destination'].replace(to_replace='La Douce',                 value='Résidences', inplace=True)
    df['destination'].replace(to_replace='La Douc',                  value='Résidences', inplace=True)
    df['destination'].replace(to_replace='Le Nôtre',                 value='Bavilliers', inplace=True)
    df['destination'].replace(to_replace='Liberté Follereau',        value='Liberté Follereau', inplace=True)
    df['destination'].replace(to_replace='LIBERTE FOLLEREAU',        value='Liberté Follereau', inplace=True)
    df['destination'].replace(to_replace='Liberté Follerea',         value='Liberté Follereau', inplace=True)
    df['destination'].replace(to_replace='Moulin',                   value='Offemont', inplace=True)
    df['destination'].replace(to_replace='OFFEMONT',                 value='Offemont', inplace=True)
    df['destination'].replace(to_replace='Offemont Cerisier',        value='Offemont', inplace=True)
    df['destination'].replace(to_replace='Offemont Cerisiers',       value='Offemont', inplace=True)
    df['destination'].replace(to_replace='OFFEMONT CERISIERS',       value='Offemont', inplace=True)
    df['destination'].replace(to_replace='Offemont Moulin',          value='Offemont', inplace=True)
    df['destination'].replace(to_replace='OFFEMONT MOULIN',          value='Offemont', inplace=True)
    df['destination'].replace(to_replace='Pierre Engel',             value='Pépinière', inplace=True)
    df['destination'].replace(to_replace='PEPINIERE',                value='Pépinière', inplace=True)
    df['destination'].replace(to_replace='Signoret',                 value='Résidences', inplace=True)
    df['destination'].replace(to_replace="Prés D'aumont",            value="Prés d'Aumont", inplace=True)
    df['destination'].replace(to_replace="Prés D'Aumont",            value="Prés d'Aumont", inplace=True)
    df['destination'].replace(to_replace="PRES D'AUMONT",            value="Prés d'Aumont", inplace=True)
    df['destination'].replace(to_replace="RESIDENCES",               value="Résidences", inplace=True)
    df['destination'].replace(to_replace="Techn'hom Cravan",         value="Cravanche", inplace=True)
    df['destination'].replace(to_replace="Techn'hom Cravanche",      value="Cravanche", inplace=True)
    df['destination'].replace(to_replace='Valdoie par Jaurès',       value='Valdoie', inplace=True)
    df['destination'].replace(to_replace='Valdoie Blumberg',         value='Valdoie', inplace=True)
    df['destination'].replace(to_replace='Valdoie Mairie',           value='Valdoie', inplace=True)
    df['destination'].replace(to_replace="VALDOIE",                  value="Valdoie", inplace=True)
    df['destination'].replace(to_replace="VALDOIE PAR JAURES",       value="Valdoie", inplace=True)
    
    df['destination'].replace(to_replace='Dépôt RTTB', value='Dépôt', inplace=True)
    df['destination'].replace(to_replace='DEPOT', value='Dépôt', inplace=True)
    
    df = df[(df['destination']!='BELFORT GARE')]
    df = df[(df['destination']!="Caisse d'Epargne")]
    df = df[(df['destination']!='Clémenceau')]
    df = df[(df['destination']!="Colbert")]
    df = df[(df['destination']!="Denfert Rocherea")]
    df = df[(df['destination']!="IUT")]
    df = df[(df['destination']!="Essert")]
    df = df[(df['destination']!="ESSERT")]
    df = df[(df['destination']!='Fg de Lyon')]
    df = df[(df['destination']!='Gare')]
    df = df[(df['destination']!='Belfort Gare')]
    df = df[(df['destination']!='Gare de Belfort')]
    df = df[(df['destination']!='GARE DE BELFORT')]
    df = df[(df['destination']!='Laurencie')]
    df = df[(df['destination']!='Lycée')]
    df = df[(df['destination']!='Lycée Quelet')]
    df = df[(df['destination']!='Lycée Quelet par Valdoie')]
    df = df[(df['destination']!='LYCEE QUELET PAR VALDOIE')]
    df = df[(df['destination']!='Mieg')]
    df = df[(df['destination']!='Oeufs Frais')]
    df = df[(df['destination']!='Préfecture')]
    df = df[(df['destination']!="Techn'hom4")]
    df = df[(df['destination']!="Liberté Follereau")]
    df = df[(df['destination']!="Liberté Madrid")]
    df = df[(df['destination']!="LIBERTE MADRID")]
    df = df[(df['destination']!="Morvillars Gare")]
    df = df[(df['destination']!="MORVILLARS GARE")]
    df = df[(df['destination']!='Quai Vauban')]
    df = df[(df['destination']!='République')]
    
    return df

In [6]:
def Update_last_stop(df):
    df['lastStop'].replace(to_replace='1ère armée',         value='1re Armée', inplace=True)
    df['lastStop'].replace(to_replace='1ère Armée',         value='1re Armée', inplace=True)
    df['lastStop'].replace(to_replace='Argiésans centre',   value='Argiésans', inplace=True)
    df['lastStop'].replace(to_replace='Bas du village',     value='Bas du Village', inplace=True)
    df['lastStop'].replace(to_replace='Benoit Frachon',     value='Benoît Frachon', inplace=True)
    df['lastStop'].replace(to_replace='Centre Commercia',   value='Centre Commercial', inplace=True)
    df['lastStop'].replace(to_replace='Chemin de la ferme', value='Chemin de la Ferme', inplace=True)
    df['lastStop'].replace(to_replace='Cimetière milita',   value='Cimetière Militaire', inplace=True)
    df['lastStop'].replace(to_replace='Cimetière militaire',value='Cimetière Militaire', inplace=True)
    df['lastStop'].replace(to_replace='Clemenceau',         value='Clémenceau', inplace=True)
    df['lastStop'].replace(to_replace='Cravanche centre',   value='Cravanche Centre', inplace=True)
    df['lastStop'].replace(to_replace='Dépôt RTTB',         value='Dépôt', inplace=True)
    df['lastStop'].replace(to_replace='Denfert Rocherea',   value='Denfert Rochereau', inplace=True)
    
#    df['lastStop'].replace(to_replace='Eglise',         value='???'', inplace=True) # Église de Grosne ou Fèche l'église ?
    df['lastStop'].replace(to_replace='Eloie Chaume',       value='Eloie', inplace=True)
    df['lastStop'].replace(to_replace='Follereau',          value='Liberté Follereau', inplace=True)
#    df['lastStop'].replace(to_replace='Haut des Prés',        value='???', inplace=True)
    df['lastStop'].replace(to_replace="Grand'combe",        value="Grand'Combe", inplace=True)
    df['lastStop'].replace(to_replace='Hôpital',            value='Hôpital Nfc', inplace=True)
    df['lastStop'].replace(to_replace='Hôpital NFC',        value='Hôpital Nfc', inplace=True)
    df['lastStop'].replace(to_replace='Jaurès',             value='Jean Jaurès', inplace=True)
    df['lastStop'].replace(to_replace='Jaurès Hôpital',     value='Jean Jaurès', inplace=True)
    df['lastStop'].replace(to_replace='Jaurès Hôptal',      value='Jean Jaurès', inplace=True)
    df['lastStop'].replace(to_replace="L'assise",           value="Froideval l'Assise", inplace=True)
    df['lastStop'].replace(to_replace='LA DAME',            value='La Dame', inplace=True)
    df['lastStop'].replace(to_replace='Les forges',         value='Les Forges', inplace=True)
    df['lastStop'].replace(to_replace='Liberté Follerea',   value='Liberté Follereau', inplace=True)
#    df['lastStop'].replace(to_replace='Les Etangs',           value='????', inplace=True)
#    df['lastStop'].replace(to_replace='Nautibus',             value='???', inplace=True)
    df['lastStop'].replace(to_replace="Prés D'Aumont",      value="Prés d'Aumont", inplace=True)
    df['lastStop'].replace(to_replace="Prés D'aumont",      value="Prés d'Aumont", inplace=True)
    df['lastStop'].replace(to_replace="Rte de Bermont",     value="Route de Bermont", inplace=True)
    df['lastStop'].replace(to_replace="Route de Sermama",   value="Route de Sermamagny", inplace=True)
    df['lastStop'].replace(to_replace="Route de Vourvenans",value="Rte de Vourvenans", inplace=True)
    df['lastStop'].replace(to_replace="Rte de Vourvenan",   value="Rte de Vourvenans", inplace=True)
#    df['lastStop'].replace(to_replace="Schweitzer",      value="???", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 1/UTBM",   value="Techn'Hom 1/UTBM", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 2",        value="Techn'Hom 2", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 3",        value="Techn'Hom 3", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom Cravan",   value="Techn'Hom Cravanche", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom Cravanche",value="Techn'Hom Cravanche", inplace=True)
    df['lastStop'].replace(to_replace="Trèvenans",          value="Trévenans", inplace=True)
    
    df = df[(df['lastStop']!='')]
    df = df[(df['lastStop']!='Dépôt')]
    df = df[(df['lastStop']!='Eglise')]
    df = df[(df['lastStop']!="Caisse d'Epargne")]
    df = df[(df['lastStop']!="Lycée")]
    df = df[(df['lastStop']!="Pont Neuf")]
    df = df[(df['lastStop']!="Techn'hom4")]
    df = df[(df['lastStop']!="Quai Vauban")]

    return df

# Save all courses from 2013 to 2020

In [ ]:
files = []
files.append(glob.glob("./csv_data/2013*.csv"))
files.append(glob.glob("./csv_data/2014*.csv"))
files.append(glob.glob("./csv_data/2015*.csv"))
files.append(glob.glob("./csv_data/2016*.csv"))
files.append(glob.glob("./csv_data/2017*.csv"))
files.append(glob.glob("./csv_data/2018*.csv"))
files.append(glob.glob("./csv_data/2019*.csv"))
files.append(glob.glob("./csv_data/2020*.csv"))


for filenames in files:
    filenames.sort()
    # concaténation et state
    df_test = concatDf(filenames)
    df_test = whitespace_remover(df_test)
    print(len(df_test))
    print(pd.unique(df_test['state'].sort_values(ascending=True)))

    # filtrage destination
    df_test = Update_final_destination(df_test)
    df_test = df_test[(df_test['destination']!='')]
    df_test = df_test[(df_test['destination']!='Dépôt')]
    dest = pd.unique(df_test['destination'].sort_values(ascending=True))
    print(np.setdiff1d(df_test["destination"], df_dest, assume_unique=False))

    # filtrage arrêts
    df_test = Update_last_stop(df_test)
    lastStop_filtered = pd.unique(df_test['lastStop'].sort_values(ascending=True))
    print(np.setdiff1d(lastStop_filtered, stop_names, assume_unique=False))

    grouped = df_test.groupby("Year")
    for name, group in grouped:
        group.to_csv("filtered_data_{}.csv".format(name))